In [ ]:
%run helperFunctions.ipynb

In [ ]:
import pandas as pd
from functools import reduce
import random

In [ ]:
precepitation = pd.read_csv('../../data/precipitation.csv')
precepitation.rename(columns={'time':'measurementDate','precipitation (mm)':'precipitation'},inplace=True)
precepitation.drop(columns=['rain (mm)'],inplace=True)
precepitation['measurementDate'] = pd.to_datetime(precepitation['measurementDate'])
precepitation.to_parquet('../../data/shiftedDates/precipitation.parquet', index=False)

In [ ]:
file_paths = [
              '../../data/shiftedDates/oxygenAB.parquet',
              '../../data/shiftedDates/cleanedNitrate.parquet',
              '../../data/cleanedData/phosphate.parquet',
              '../../data/shiftedDates/cleanedAmmonium.parquet',
              '../../data/shiftedDates/cleanedTotal.parquet',
              '../../data/shiftedDates/precipitation.parquet'
              ]

dfs = [pd.read_parquet(path) for path in file_paths]

In [ ]:

merged = reduce(lambda left, right: pd.merge(left, right, how='outer', on='measurementDate'), dfs)
merged['phosphateValue'] = merged['phosphateValue'].map(lambda phosphate: -999 if pd.isnull(phosphate) else phosphate)

In [ ]:
precepitationIndex=0
for row in merged['precipitation']:
    if not pd.isnull(row):
        # print(row)
        valuePerMinute = row/60
        if precepitationIndex < 60:
            merged['precipitation'][:precepitationIndex] = valuePerMinute
        else:
            merged['precipitation'][precepitationIndex-60:precepitationIndex] = valuePerMinute
    precepitationIndex+=1



In [ ]:
# for percipitation in df['percipiation']
#  if percipitation != null
#     val = percipitoan/60
#     df['precipitation'][precitpitation.index-60:precepitation.index] = val


In [ ]:
merged.to_parquet('../../data/cleanedData/allData.parquet')